# Data Preprocessing

This notebook is dedicated to the preprocessing of **player-level seasonal dataset**.   

The starting point consists of already constructed raw datasets, one for each competition. These datasets will be aggregated and preprocessed to obtain a unified structure that can be consistently used for subsequent analysis and modeling.  

The workflow will be organized into modular steps to ensure clarity and reproducibility. Each step will progressively refine the data into a form suitable for the next stage of the project.


## Imports

In [70]:
import pandas as pd
import ast 
import numpy as np
import os

from tqdm import tqdm
from statsbombpy import sb
from collections import Counter
from IPython.display import display

import warnings
warnings.filterwarnings("ignore")

## Aggregate raw datasets

The first step consists of loading the raw player datasets that were previously constructed for each competition. These files will be concatenated into a single unified dataset, preserving the information on the corresponding season.

In [71]:
df_premier = pd.read_csv("../task2_ballon_dor/data/premier_league_2015_16.csv")
df_liga = pd.read_csv("../task2_ballon_dor/data/laliga_2015_16.csv")
df_serieA = pd.read_csv("../task2_ballon_dor/data/serieA_2015_16.csv")
df_bundesliga = pd.read_csv("../task2_ballon_dor/data/bundesliga_2015_16.csv")
df_ligue1 = pd.read_csv("../task2_ballon_dor/data/ligue1_2015_16.csv")

df_final = pd.concat([df_premier, df_liga, df_serieA, df_bundesliga, df_ligue1], ignore_index=True).reset_index(drop=True)

print("Final dataset shape:", df_final.shape)

print("\nColumns:", df_final.columns)

print("\nCompetitions in dataset:", df_final['competition'].unique())

Final dataset shape: (2687, 48)

Columns: Index(['competition', 'season', 'teams', 'main_role', 'player_id',
       'player_name', 'presences', 'matches_started', 'full_matches',
       'minutes_played', 'substitutions_in', 'substitutions_out',
       'yellow_cards', 'red_cards', 'shots_attempted', 'shots_on_target',
       'goals', 'xg_total', 'assists', 'key_passes', 'passes_attempted',
       'passes_completed', 'progressive_passes', 'crosses', 'switches',
       'carries_attempted', 'carry_distance_total', 'progressive_carries',
       'carries_to_penalty_area', 'dribbles_attempted', 'dribbles_completed',
       'duels_attempted', 'duels_won', 'interceptions_attempted',
       'interceptions_won', 'blocks', 'clearances', 'ball_recoveries',
       'pressures', 'dispossessed', 'fouls_committed', 'fouls_won',
       'own_goals', 'gk_goals_conceded', 'gk_shots_faced', 'gk_saves',
       'gk_penalties_saved', 'gk_clean_sheet'],
      dtype='object')

Competitions in dataset: ['England -

## Identify players with multiple teams (same competition)

In some cases, a player may appear for more than one team during the same season (e.g., due to transfers or loans). The `teams` column initially stores values as strings that look like lists (e.g., `"['AFC Bournemouth', 'Reading']"`).  

In [72]:
# Convert "teams" column into Python lists
df_final["teams"] = df_final["teams"].apply(
    lambda x: ast.literal_eval(x) if isinstance(x, str) else x
)

# Filter players with more than one team
multi_team_df = df_final[df_final["teams"].apply(lambda x: len(x) > 1)]

# Drop duplicates just in case
multi_team_df = multi_team_df[["player_name", "teams"]]

print("Players with more than 1 team in the season:\n")
for _, row in multi_team_df.iterrows():
    print(f"- {row['player_name']}: {', '.join(row['teams'])}")


Players with more than 1 team in the season:

- Glenn Murray: AFC Bournemouth, Crystal Palace
- Lewis Grabban: AFC Bournemouth, Norwich City
- Joleon Lescott: Aston Villa, West Bromwich Albion
- Nathan Dyer: Leicester City, Swansea City
- Matt Jarvis: Norwich City, West Ham United
- Steven Naismith: Everton, Norwich City
- Patrick Bamford: Crystal Palace, Norwich City
- Steven Caulker: Liverpool, Southampton
- Jonjo Shelvey: Newcastle United, Swansea City
- Andros Townsend: Newcastle United, Tottenham Hotspur
- Alex Pritchard: Tottenham Hotspur, West Bromwich Albion
- Raúl García: Athletic Club, Atlético Madrid
- Augusto Fernández: Atlético Madrid, Celta Vigo
- Lucas Orbán: Levante UD, Valencia
- Chory Castro: Málaga, Real Sociedad
- Piti: Granada, Rayo Vallecano
- Guilherme Siqueira: Atlético Madrid, Valencia
- Denis Cheryshev: Real Madrid, Valencia
- Juraj Kucka: AC Milan, Genoa
- Alessio Cerci: AC Milan, Genoa
- Serge Gakpé: Atalanta, Genoa
- Marco Borriello: Atalanta, Carpi
- Matte

## Manage duplicated players across rows (transfers in different competition)


In the previous step, the focus was on the `teams` column to detect players who had represented more than one club **within the same competition**. In that case, transfers or loans were already merged into a single season record (e.g., `teams = ["Liverpool", "Bolton"]`).  

Here, the attention shifts to players who appear in multiple rows of the dataset with the same `player_id`. This situation usually arises when a player changes club **and also moves across different competitions**, leading to multiple season entries rather than a single aggregated one.

In [73]:
# Find duplicated players (same player_id appearing in multiple rows)
dup_players = df_final[df_final.duplicated(subset=["player_id"], keep=False)]

# Keep only relevant info
dup_players = dup_players[["player_id", "player_name", "teams"]]

# Group by player_id to see all teams per player
dup_grouped = dup_players.groupby(["player_id", "player_name"])["teams"].apply(list).reset_index()

count = 0

# Print results
print("Players appearing in multiple rows:\n")
for _, row in dup_grouped.iterrows():
    all_teams = [", ".join(t) if isinstance(t, list) else str(t) for t in row["teams"]]
    print(f"- {row['player_name']}: {all_teams}")
    count += 1

print(f"Total duplicated players found: {count}")


Players appearing in multiple rows:

- Javier Hernández Balcázar: ['Manchester United', 'Bayer Leverkusen']
- Son Heung-Min: ['Tottenham Hotspur', 'Bayer Leverkusen']
- Kevin De Bruyne: ['Manchester City', 'Wolfsburg']
- Milan Biševac: ['Lazio', 'Lyon']
- Wahbi Khazri: ['Sunderland', 'Bordeaux']
- Sergi Darder: ['Málaga', 'Lyon']
- Florian Thauvin: ['Newcastle United', 'Marseille']
- Abdoulaye Doucouré: ['Granada', 'Rennes']
- Nordin Amrabat: ['Watford', 'Málaga']
- Fernando Llorente: ['Sevilla', 'Juventus']
- Mario Lemina: ['Juventus', 'Marseille']
- Ola Toivonen: ['Sunderland', 'Rennes']
- Mathieu Debuchy: ['Arsenal', 'Bordeaux']
- Henri Saivet: ['Newcastle United', 'Bordeaux']
- Andrej Kramarić: ['Leicester City', 'Hoffenheim']
- Ivan Perišić: ['Inter Milan', 'Wolfsburg']
- Takashi Inui: ['Eibar', 'Eintracht Frankfurt']
- Juan Cuadrado: ['Chelsea', 'Juventus']
- Adnan Januzaj: ['Manchester United', 'Borussia Dortmund']
- Martín Montoya: ['Real Betis', 'Inter Milan']
- Claudio Beauvu

#### Fix "David N'Gog"

In [74]:
# Fix player name variants for David N'Gog
variants = ["David N'Gog", 'David N"Gog']

# Subset of the dataframe with the variants
subset = df_final[df_final["player_name"].isin(variants)]

# Show BEFORE
print("BEFORE")
print(subset[["player_name", "presences", "goals", "assists", "xg_total", "minutes_played"]])

if len(subset) > 1:
    # Sum only the numeric columns
    summed_stats = subset.select_dtypes(include=["number"]).sum(numeric_only=True)

    # Take the index of the first row you want to keep
    keep_idx = subset.index[0]

    # Update the numeric values of the kept row
    for col, val in summed_stats.items():
        df_final.at[keep_idx, col] = val

    # Force the canonical name
    df_final.at[keep_idx, "player_name"] = "David N'Gog"

    # Drop the other duplicate rows
    drop_idx = subset.index.drop(keep_idx)
    df_final = df_final.drop(index=drop_idx)

    # Show AFTER
    print("\nAFTER")
    print(df_final.loc[[keep_idx], ["player_name", "presences", "goals", "assists", "xg_total", "minutes_played"]])
else:
    print("\nNo variants found for David N'Gog")


BEFORE
      player_name  presences  goals  assists  xg_total  minutes_played
2628  David N"Gog         15      3        0      3.31             824
2629  David N'Gog          1      0        0      0.11              76

AFTER
      player_name  presences  goals  assists  xg_total  minutes_played
2628  David N'Gog         16      3        0      3.42             900


#### Merge duplicated players across competitions

When the same player appears in multiple rows within a season, this indicates that they played for more than one team in more than one competition.   To prepare a clean season-level dataset, these duplicates need to be merged into a single row.

The merging rules are:

- **Numeric statistics** → summed across all rows (e.g., goals, assists, minutes played).  

- **Teams** → combined into a unique list of all clubs represented by the player.  

- **Competitions** → combined into a list of all competitions in which the player appeared.  

- **Metadata** (`player_id`, `player_name`, `season`) → kept consistent.  

This approach ensures that each player has exactly **one record per season**, which reflects their overall performance across different teams and competitions.


In [75]:
def choose_main_role(roles, minutes):
    """
    Choose main role:
    - Most frequent role across the season
    - If tie between roles -> take the one with more minutes played
    """
    role_list = list(roles)
    minutes_list = list(minutes)

    # Frequencies of roles
    counts = Counter(role_list)
    most_common = counts.most_common()
    top_count = most_common[0][1]

    # Roles with top frequency
    tied_roles = [r for r, c in most_common if c == top_count]

    # If only one role is tied for most frequent, return it
    if len(tied_roles) == 1:
        return tied_roles[0]
    else:
        # Tie: select the role with more minutes
        role_minutes = {r: 0 for r in tied_roles}
        for r, m in zip(role_list, minutes_list):
            if r in role_minutes:
                role_minutes[r] += m
        return max(role_minutes.items(), key=lambda x: x[1])[0]

In [76]:
def merge_duplicate_players(df: pd.DataFrame):
    """
    Merge players with multiple rows in the same season into a single row.

    Inspired by collect_player_season_stats:
      - Numeric stats are summed (default)
      - Average-type columns are averaged
      - Teams are aggregated as unique list
      - Competitions aggregated as unique list
      - Main role: most frequent, tie broken by minutes played
      - Metadata (season, player_id, player_name) preserved
    """

    # Metadata
    meta_cols = ["season", "player_id", "player_name"]

    # Numeric cols
    numeric_cols = df.select_dtypes(include=["number"]).columns.tolist()
    agg_dict = {col: "sum" for col in numeric_cols if col not in meta_cols and col not in ["teams", "competition"]}

    # Start aggregation step (numeric first)
    df_merged = df.groupby(meta_cols).agg(agg_dict).reset_index()

    # Teams (unique list)
    df_teams = (
        df.groupby(meta_cols)["teams"]
        .apply(lambda x: sorted(set(sum(x, []))))
        .reset_index()
    )
    df_merged = df_merged.merge(df_teams, on=meta_cols, how="left")

    # Competitions (unique list)
    df_comp = (
        df.groupby(meta_cols)["competition"]
        .apply(lambda x: sorted(set(x)))
        .reset_index()
        .rename(columns={"competition": "competitions"})
    )
    df_merged = df_merged.merge(df_comp, on=meta_cols, how="left")

    # Main role (most frequent, tie-break with minutes_played)
    df_roles = (
        df.groupby(meta_cols)
        .apply(lambda g: choose_main_role(g["main_role"], g["minutes_played"]))
        .reset_index(name="main_role")
    )

    df_merged = df_merged.merge(df_roles, on=meta_cols, how="left")

    return df_merged


In [77]:
# Example: check a player (Stephan El Shaarawy, half season at AS Monaco and half season at AS Roma)
print("BEFORE:")
print(df_final[df_final["player_name"] == "Stephan El Shaarawy"][["player_name", "presences", "goals", "assists", "xg_total", "minutes_played"]])
print("Shape:", df_final.shape)

BEFORE:
              player_name  presences  goals  assists  xg_total  minutes_played
1140  Stephan El Shaarawy         16      8        2      6.16            1228
2137  Stephan El Shaarawy         15      0        0      1.82             730
Shape: (2686, 48)


In [78]:
# Merge duplicate players
df_final = merge_duplicate_players(df_final)

print("AFTER:")
print(df_final[df_final["player_name"] == "Stephan El Shaarawy"][["player_name", "presences", "goals", "assists", "xg_total", "minutes_played"]])
print("Shape:", df_final.shape)

df_final[df_final["player_name"] == "Stephan El Shaarawy"]

AFTER:
              player_name  presences  goals  assists  xg_total  minutes_played
1052  Stephan El Shaarawy         31      8        2      7.98            1958
Shape: (2635, 48)


,season,player_id,player_name,presences,matches_started,full_matches,minutes_played,substitutions_in,substitutions_out,yellow_cards,...,fouls_won,own_goals,gk_goals_conceded,gk_shots_faced,gk_saves,gk_penalties_saved,gk_clean_sheet,teams,competitions,main_role
1052,2015/2016,6979,Stephan El Shaarawy,31,22,3,1958,9,19,1,...,14,0,0,0,0,0,0,"[AS Monaco, AS Roma]","[France - Ligue 1, Italy - Serie A]",Left Wing


## Team League Position

Individual performance must also be interpreted in the context of **team success**. For this reason, each player will be linked to the final league position of their team in the corresponding season.  

Since this information is not included in the raw event data, a separate reference table with team standings per season is manually created and then merged with the player dataset.

> **NOTE**: if a player played for different teams, a list with the final league position for each team is stored in the player dataset

In [79]:
def build_league_table(competition_id, season_id, competition_name):
    """
    Build final league table for a given competition and season.
    """
    # Load all matches
    matches = sb.matches(competition_id=competition_id, season_id=season_id)
    
    # Initialize standings
    standings = {}

    # Iterate matches
    for _, match in tqdm(matches.iterrows(), total=matches.shape[0], desc=f"Processing {competition_name}"):
        home = match["home_team"]
        away = match["away_team"]
        home_goals = match["home_score"]
        away_goals = match["away_score"]

        # Ensure teams are in standings dict
        if home not in standings:
            standings[home] = {"points": 0, "gf": 0, "ga": 0}
        if away not in standings:
            standings[away] = {"points": 0, "gf": 0, "ga": 0}

        # Update goals
        standings[home]["gf"] += home_goals
        standings[home]["ga"] += away_goals
        standings[away]["gf"] += away_goals
        standings[away]["ga"] += home_goals

        # Assign points
        if home_goals > away_goals:
            standings[home]["points"] += 3
        elif home_goals < away_goals:
            standings[away]["points"] += 3
        else:
            standings[home]["points"] += 1
            standings[away]["points"] += 1

    # Convert to DataFrame
    table = pd.DataFrame([
        {
            "team": team,
            "points": data["points"],
            "goals_for": data["gf"],
            "goals_against": data["ga"],
            "goal_difference": data["gf"] - data["ga"]
        }
        for team, data in standings.items()
    ])

    # Rank teams by points, goal difference, goals for
    table = table.sort_values(
        by=["points", "goal_difference", "goals_for"], 
        ascending=[False, False, False]
    ).reset_index(drop=True)

    # Add final position as first column
    table.insert(0, "team_league_position", table.index + 1)

    return table


In [80]:
# Dictionary of competition_id for Big 5
competitions_2015_16 = {
    "England - Premier League": {"competition_id": 2, "season_id": 27},
    "Spain - La Liga": {"competition_id": 11, "season_id": 27},
    "Germany - 1. Bundesliga": {"competition_id": 9, "season_id": 27},
    "France - Ligue 1": {"competition_id": 7, "season_id": 27},
    "Italy - Serie A": {"competition_id": 12, "season_id": 27}
}

# Build tables for all
league_tables = {}
for comp_name, ids in competitions_2015_16.items():
    print(f"Building table for {comp_name}...")
    table = build_league_table(ids["competition_id"], ids["season_id"], comp_name)
    league_tables[comp_name] = table

Building table for England - Premier League...


Processing England - Premier League: 100%|██████████| 380/380 [00:00<00:00, 15768.84it/s]


Building table for Spain - La Liga...


Processing Spain - La Liga: 100%|██████████| 380/380 [00:00<00:00, 28601.80it/s]


Building table for Germany - 1. Bundesliga...


Processing Germany - 1. Bundesliga: 100%|██████████| 306/306 [00:00<00:00, 27552.64it/s]


Building table for France - Ligue 1...


Processing France - Ligue 1: 100%|██████████| 377/377 [00:00<00:00, 27657.82it/s]

Building table for Italy - Serie A...



Processing Italy - Serie A: 100%|██████████| 380/380 [00:00<00:00, 28183.54it/s]


> **NOTE**: he league tables are not fully accurate for Ligue 1 because three matches are missing from the dataset. However, this does not affect the key outcomes: PSG remains 1st, Ajaccio remains 19th, and Bordeaux is listed as 14th instead of 11th.

In [81]:
# Print all the final league tables nicely
for comp_name, table in league_tables.items():
    print(f"Final table for {comp_name}:\n")
    display(table)
    print("\n")

Final table for England - Premier League:



,team_league_position,team,points,goals_for,goals_against,goal_difference
0,1,Leicester City,81,68,36,32
1,2,Arsenal,71,65,36,29
2,3,Tottenham Hotspur,70,69,35,34
3,4,Manchester City,66,71,41,30
4,5,Manchester United,66,49,35,14
5,6,Southampton,63,59,41,18
6,7,West Ham United,62,65,51,14
7,8,Liverpool,60,63,50,13
8,9,Stoke City,51,41,55,-14
9,10,Chelsea,50,59,53,6




Final table for Spain - La Liga:



,team_league_position,team,points,goals_for,goals_against,goal_difference
0,1,Barcelona,91,112,29,83
1,2,Real Madrid,90,110,34,76
2,3,Atlético Madrid,88,63,18,45
3,4,Villarreal,64,44,35,9
4,5,Athletic Club,62,58,45,13
5,6,Celta Vigo,60,51,59,-8
6,7,Sevilla,52,51,50,1
7,8,Málaga,48,38,35,3
8,9,Real Sociedad,48,45,48,-3
9,10,Real Betis,45,34,52,-18




Final table for Germany - 1. Bundesliga:



,team_league_position,team,points,goals_for,goals_against,goal_difference
0,1,Bayern Munich,88,80,17,63
1,2,Borussia Dortmund,78,82,34,48
2,3,Bayer Leverkusen,60,56,40,16
3,4,Borussia Mönchengladbach,55,67,50,17
4,5,Schalke 04,52,51,49,2
5,6,FSV Mainz 05,50,46,42,4
6,7,Hertha Berlin,50,42,42,0
7,8,Wolfsburg,45,47,49,-2
8,9,FC Köln,43,38,42,-4
9,10,Hamburger SV,41,40,46,-6




Final table for France - Ligue 1:



,team_league_position,team,points,goals_for,goals_against,goal_difference
0,1,Paris Saint-Germain,93,100,19,81
1,2,Lyon,65,67,43,24
2,3,AS Monaco,65,57,50,7
3,4,OGC Nice,63,58,41,17
4,5,Lille,60,39,27,12
5,6,Saint-Étienne,58,42,35,7
6,7,Caen,54,39,52,-13
7,8,Rennes,52,52,54,-2
8,9,Angers,50,40,38,2
9,10,Bastia,50,35,40,-5




Final table for Italy - Serie A:



,team_league_position,team,points,goals_for,goals_against,goal_difference
0,1,Juventus,91,75,20,55
1,2,Napoli,82,80,32,48
2,3,AS Roma,80,83,41,42
3,4,Inter Milan,67,50,38,12
4,5,Fiorentina,64,60,42,18
5,6,Sassuolo,61,49,40,9
6,7,AC Milan,57,49,43,6
7,8,Lazio,54,52,52,0
8,9,Chievo,50,43,45,-2
9,10,Genoa,46,45,48,-3


In [82]:
# Assign league position to players
def assign_team_positions(df_players, league_tables):
    """
    Merge league table info into player dataset.
    
    - If player has 1 team in 1 competition => assign its league position (int)
    - If player has multiple teams in 1 competition => assign list of positions
    - If player has multiple competitions => assign list of positions
    """
    results = []

    for _, row in tqdm(df_players.iterrows(), total=df_players.shape[0]):
        comps = row["competitions"]
        teams = row["teams"]

        # Ensure always list
        if isinstance(comps, str):
            comps = [comps]
        if isinstance(teams, str):
            teams = [teams]

        # Handle multi-competition
        if len(comps) > 1:
            positions = []
            # Iterate over competitions
            for comp in comps:

                # Get league table for competition
                table = league_tables[comp]

                # Iterate over teams
                for t in teams:
                    if t in table["team"].values:
                        # Get position for team
                        pos = table.loc[table["team"] == t, "team_league_position"].values[0]
                        positions.append(int(pos))
            results.append(positions if positions else None)

        else:
            # Single competition
            comp = comps[0]
            positions = []
            table = league_tables[comp]

            # Iterate over teams
            for t in teams:
                # Get position for team
                pos = table.loc[table["team"] == t, "team_league_position"].values[0]
                positions.append(int(pos))

            # Store single value or list
            if len(positions) == 1:
                results.append(positions[0])
            else:
                results.append(positions)

    df_players["team_league_position"] = results
    return df_players


In [83]:
df_final = assign_team_positions(df_final, league_tables)
df_final[["player_name", "teams", "team_league_position"]].sample(10)

100%|██████████| 2635/2635 [00:00<00:00, 5010.72it/s]


,player_name,teams,team_league_position
936,Mehdi Lacen,[Getafe],19
675,Raphaël Varane,[Real Madrid],2
796,Michael Krohn-Dehli,[Sevilla],7
425,Calum Chambers,[Arsenal],2
184,Marouane Fellaini,[Manchester United],5
1332,Maxi López,[Torino],12
158,Hamari Traoré,[Stade de Reims],18
254,Alexander Söderlund,[Saint-Étienne],6
1401,Mark Uth,[Hoffenheim],15
1555,Przemysław Tytoń,[VfB Stuttgart],17


In [84]:
# Check with El Shaarawy that changed competitions
df_final[df_final["player_name"] == "Stephan El Shaarawy"]

,season,player_id,player_name,presences,matches_started,full_matches,minutes_played,substitutions_in,substitutions_out,yellow_cards,...,own_goals,gk_goals_conceded,gk_shots_faced,gk_saves,gk_penalties_saved,gk_clean_sheet,teams,competitions,main_role,team_league_position
1052,2015/2016,6979,Stephan El Shaarawy,31,22,3,1958,9,19,1,...,0,0,0,0,0,0,"[AS Monaco, AS Roma]","[France - Ligue 1, Italy - Serie A]",Left Wing,"[3, 3]"


## Add macro_role 

While the dataset contains detailed positional information for each player (e.g., *Left Back*, *Central Midfielder*, *Right Winger*), these roles are often too granular for higher-level comparisons and ranking purposes.  

To simplify the analysis and ensure consistency across different teams and competitions, each role will be mapped into a broader **macro-role** category:

- **GK** → Goalkeepers  

- **DEF** → Defenders (full-backs, center-backs, wing-backs)  

- **MID** → Midfielders (defensive, central, attacking)  

- **ATT** → Attackers (wingers, forwards, strikers)

The mapping between detailed roles and macro-roles is provided in the previous computed file `../task1_xg/data/unique_roles.csv`. 

> **NOTE**: However the code to create the file and save it in the first task folder is left below and is launched only if the file is not found.

This file will be loaded and used to enrich the player dataset with a new column `macro_role`, which groups players into these four simplified categories.

In [85]:
# Path where the roles file will be saved
roles_file = "../task1_xg/data/unique_roles.csv"

# Check if the file already exists
if os.path.exists(roles_file):
    print(f"File '{roles_file}' already exists. Skipping extraction.")
    roles_df = pd.read_csv(roles_file)
else:
    # Container for unique player roles
    all_roles = set()
    skipped_matches = 0

    # Load all available competitions from StatsBomb and sort them by season
    comps = sb.competitions().sort_values(by="season_name")

    # Iterate over competitions with a progress bar
    for _, comp in tqdm(comps.iterrows(), total=len(comps), desc="Competitions"):
        comp_id = comp["competition_id"]
        season_id = comp["season_id"]

        # Load all matches for the given competition and season
        matches = sb.matches(competition_id=comp_id, season_id=season_id)

        # Iterate over all matches (no progress bar to keep it lighter)
        for match_id in matches["match_id"].unique():

            # Load event data for the match
            events = sb.events(match_id=match_id)

            # Filter Starting XI events
            starting_xi = events[events["type"] == "Starting XI"]

            # For each Starting XI (one per team), extract player roles
            for i in range(len(starting_xi)):
                lineup = starting_xi.iloc[i]["tactics"].get("lineup", [])
                for player in lineup:
                    role = player.get("position", {}).get("name")
                    if role:
                        all_roles.add(role)


    # Save all unique roles into a DataFrame
    roles_df = pd.DataFrame(sorted(all_roles), columns=["role"])

    print(f"\nFile '{roles_file}' saved with", len(roles_df), "unique roles.")


File '../task1_xg/data/unique_roles.csv' already exists. Skipping extraction.


In [86]:
# Map main roles to macro roles

# Load mapping of unique roles
roles_map = pd.read_csv("../task1_xg/data/unique_roles.csv")

# Ensure consistent column names
roles_map = roles_map.rename(columns={
    "role": "main_role",         # detailed role from dataset
    "macro_role": "macro_role"   # broad role bucket
})

# Merge with players dataset
df_final = df_final.merge(
    roles_map[["main_role", "macro_role"]],
    on="main_role",
    how="left"
)

# Check mapping
df_final[["player_name", "main_role", "macro_role"]].sample(10)

,player_name,main_role,macro_role
1136,Darko Lazović,Right Wing,FWD
890,Igor Zubeldía,Left Defensive Midfield,MID
2016,Christian Träsch,Right Back,DEF
134,Dante,Left Center Back,DEF
2457,Dušan Veškovac,Right Center Back,DEF
1980,Nicolás Castillo,Left Center Forward,FWD
119,Marco Verratti,Right Center Midfield,MID
741,Salif Sané,Left Defensive Midfield,MID
1217,Luca Rigoni,Right Center Midfield,MID
451,Angelo Ogbonna,Left Center Back,DEF


In [87]:
# Check for any unmapped roles
print(len(df_final[df_final["macro_role"].isna()]))

# Print the player, team, presence and minutes of any unmapped roles
print(df_final[df_final["main_role"] == "Unknown"][["player_name", "teams", "presences", "minutes_played"]])

# Handle for any unmapped roles
df_final["macro_role"] = df_final["macro_role"].fillna("Unknown")

3
           player_name            teams  presences  minutes_played
2012  Michael Zetterer  [Werder Bremen]          0               0
2400           Rubinho       [Juventus]          0               0
2528       Ivan Kelava        [Granada]          0               0


In [88]:
# Delete rows where macro_role is "Unknown"
df_final = df_final[df_final["macro_role"] != "Unknown"]

df_final.shape

(2632, 50)

## Compute ratios and accuracy metrics

In this step, additional features are derived from the raw event-based counts.  
The following metrics will be computed and added to the dataset:  

- **Passing** → `pass_accuracy`  

- **Dribbling** → `dribble_success_rate`  

- **Duels** → `duels_lost`, `duels_ratio`  

- **Interceptions** → `interceptions_lost`, `interceptions_ratio`  

- **Goalkeeping** → `gk_save_ratio`  

- **Discipline** → `fouls_balance`


In [89]:
# Make a copy of the final dataframe to work with
df = df_final.copy()
df.shape

(2632, 50)

In [90]:
# Insert after passes_completed
insert_pos = df.columns.get_loc("passes_completed") + 1

# Pass Accuracy (divide by zero handling)
df.insert(
    insert_pos,
    "pass_accuracy",
    df["passes_completed"].div(df["passes_attempted"]).fillna(0)
)

In [91]:
# Insert after dribbles_completed
insert_pos = df.columns.get_loc("dribbles_completed") + 1

# Dribbles Success Rate
df.insert(
    insert_pos,
    "dribbles_success_rate",
    df["dribbles_completed"].div(df["dribbles_attempted"]).fillna(0)
)


In [92]:
# Duels (lost + ratio, insert after duels_won)
insert_pos = df.columns.get_loc("duels_won") + 1

# Duels Lost
df.insert(insert_pos, "duels_lost",
          df["duels_attempted"] - df["duels_won"])

# Duels Ratio
df.insert(
    insert_pos + 2,
    "duels_success_rate",
    df["duels_won"].div(df["duels_attempted"]).fillna(0)
)

In [93]:
# Insert after interceptions_won
insert_pos = df.columns.get_loc("interceptions_won") + 1

# Interceptions Lost
df.insert(insert_pos, "interceptions_lost",
          df["interceptions_attempted"] - df["interceptions_won"])

# Interceptions Success Rate
df.insert(
    insert_pos + 2, 
    "interceptions_ratio",
    df["interceptions_won"].div(df["interceptions_attempted"]).fillna(0)
)

In [94]:
# Insert after gk_saves
insert_pos = df.columns.get_loc("gk_saves") + 1

# Goalkeeper Save Ratio
df.insert(
    insert_pos + 1,
    "gk_save_ratio",
    df["gk_saves"].div(df["gk_shots_faced"]).fillna(0)
)

In [95]:
# Insert after fouls_won
insert_pos = df.columns.get_loc("fouls_won") + 1

# Fouls Balance
df.insert(insert_pos, "fouls_balance",
          df["fouls_won"] - df["fouls_committed"])

In [96]:
print("Dataset shape after feature engineering:", df.shape)
df.head()

Dataset shape after feature engineering: (2632, 58)


,season,player_id,player_name,presences,matches_started,full_matches,minutes_played,substitutions_in,substitutions_out,yellow_cards,...,gk_shots_faced,gk_saves,gk_penalties_saved,gk_save_ratio,gk_clean_sheet,teams,competitions,main_role,team_league_position,macro_role
0,2015/2016,2936,Christophe Kerbrat,29,29,28,2755,0,1,7,...,0,0,0,0.000000,0,[Guingamp],[France - Ligue 1],Right Center Back,16,DEF
1,2015/2016,2943,Lucas Deaux,16,13,10,1266,3,3,3,...,0,0,0,0.000000,0,[Nantes],[France - Ligue 1],Right Defensive Midfield,13,MID
2,2015/2016,2944,Benjamin Corgnet,16,5,1,613,11,4,0,...,0,0,0,0.000000,0,[Saint-Étienne],[France - Ligue 1],Center Attacking Midfield,6,MID
3,2015/2016,2946,Frédéric Guilbert,29,28,26,2574,1,2,3,...,0,0,0,0.000000,0,[Bordeaux],[France - Ligue 1],Right Center Back,14,DEF
4,2015/2016,2947,Anthony Lopes,37,37,37,3548,0,0,2,...,213,105,0,0.492958,13,[Lyon],[France - Ligue 1],Goalkeeper,2,GK


In [97]:
df_final = df.copy()

## Add per90 metrics

Raw counts of player actions can be heavily influenced by playing time: players who accumulate more minutes will naturally record higher totals, which makes it difficult to compare them fairly with players who played fewer minutes.  

To address this issue, selected performance metrics are normalized **per 90 minutes** of play. This allows comparisons on an equal footing.

The general formula is:

$
\text{metric\_per90} = \frac{\text{raw\_metric}}{\text{minutes\_played} / 90}
$

Not all statistics are suitable for per-90 normalization.  
Ler's apply it only to a **subset of key performance actions**, which are the most representative for evaluating player contributions:

- **Attacking**: `shots_attempted`, `shots_on_target`, `goals`, `xg_total`, `assists`, `key_passes`, `carries_attempted`, `carry_distance_total`, `carries_to_penalty_area`, `dribbles_attempted`, `dribbles_completed`  

- **Passing**: `passes_attempted`, `passes_completed`, `progressive_passes`, `crosses`

- **Defensive actions**: `duels_attempted`, `duels_won`, `interceptions_attempted`, `interceptions_won`, `blocks`, `clearances`, `ball_recoveries`, `pressures`, `fouls_committed`  

- **Goalkeeping**: `gk_shots_faced`, `gk_saves`, `gk_goals_conceded`

In [98]:
# Compute per-90 metrics
df = df_final.copy()

# Metrics selected for per90 normalization
per90_metrics = [
    # Attacking
    "shots_attempted", "shots_on_target", "goals", "xg_total",
    "assists", "key_passes", "carries_attempted", "carry_distance_total",
    "carries_to_penalty_area", "dribbles_attempted", "dribbles_completed",

    # Passing
    "passes_attempted", "passes_completed", "progressive_passes", "crosses",

    # Defensive actions
    "duels_attempted", "duels_won", "interceptions_attempted", "interceptions_won",
    "blocks", "clearances", "ball_recoveries", "pressures", "fouls_committed",

    # Goalkeeping
    "gk_shots_faced", "gk_saves", "gk_goals_conceded"
]

# Compute per90 for each metric and insert next to original column
for col in per90_metrics:
    if col in df.columns:
        per90_col = f"{col}_per90"
        insert_pos = df.columns.get_loc(col) + 1

        df.insert(
            insert_pos,
            per90_col,
            df[col] / (df["minutes_played"].replace(0, np.nan) / 90)
        )

        # round
        df[per90_col] = df[per90_col].round(2)

# Check result
df[[c for c in df.columns if "per90" in c]].head()

,shots_attempted_per90,shots_on_target_per90,goals_per90,xg_total_per90,assists_per90,key_passes_per90,passes_attempted_per90,passes_completed_per90,progressive_passes_per90,crosses_per90,...,interceptions_attempted_per90,interceptions_won_per90,blocks_per90,clearances_per90,ball_recoveries_per90,pressures_per90,fouls_committed_per90,gk_goals_conceded_per90,gk_shots_faced_per90,gk_saves_per90
0,0.13,0.00,0.00,0.01,0.00,0.13,36.16,28.49,12.54,0.13,...,1.83,0.49,1.70,5.78,3.69,11.21,0.85,0.00,0.0,0.00
1,0.28,0.07,0.00,0.02,0.00,0.36,44.15,35.83,13.51,0.43,...,0.85,0.50,2.49,2.27,3.91,21.04,1.85,0.00,0.0,0.00
2,1.62,0.29,0.15,0.17,0.15,1.03,38.76,23.49,7.05,1.32,...,0.88,0.44,1.76,1.03,4.99,21.29,2.64,0.00,0.0,0.00
3,0.35,0.03,0.00,0.01,0.00,0.24,43.22,32.52,11.71,0.84,...,1.75,0.42,2.20,3.88,3.85,13.95,1.01,0.00,0.0,0.00
4,0.00,0.00,0.00,0.00,0.00,0.00,27.02,19.28,16.69,0.00,...,0.00,0.00,0.05,0.10,4.21,0.10,0.00,1.09,5.4,2.66


In [99]:
df_final = df.copy()

## Data quality checks

In [100]:
print("Missing values per column:")
missing = df_final.isnull().sum()
missing = missing[missing > 0].sort_values(ascending=False)
print(missing)

Missing values per column:
Series([], dtype: int64)


## Save df_final dataset

In [101]:
# Save the dataset
df_final.to_csv("../task2_ballon_dor/data/df_final.csv", index=False)